## 라이브러리 import 및 설정

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords

from pathlib import Path
import warnings

In [3]:
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 학습데이터 로드

In [4]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 2020 

In [5]:
algo_name = 'lr'
feature_name = 'tfidf-pca' #학습할 피처- tfidf 피처를 pca한 데이터
feature_target = 'feature_target'

model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv' #학습할 피처 가져오기
feature_target_file = feature_dir / f'{feature_target}.csv' #target 데이터 가져오기

p_val_ver7_file = val_dir / f'{model_name}_oof_pred_ver7.csv'
p_tst_ver7_file = tst_dir / f'{model_name}_test_pred_ver7.csv'
sub_ver7_file = sub_dir / f'{model_name}_ver7.csv'

## 데이터 셋 불러오기

In [6]:
X = pd.read_csv(feature_file, index_col=0).values
y = pd.read_csv(feature_target_file,index_col=0).values

X_7 = X[:54879]
X_tst_7 = X[54879:]

In [7]:
print(X_7.shape)
print(X_tst_7.shape)
print(type(X_7))

(54879, 500)
(19617, 500)
<class 'numpy.ndarray'>


## 로지스틱회귀 모델 학습

In [8]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [9]:
p_val_ver7 = np.zeros((X_7.shape[0], n_class))
p_tst_ver7 = np.zeros((X_tst_7.shape[0], n_class))


for X, test in [(X_7, X_tst_7)]:  
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        X_train, X_val = X[i_trn], X[i_val]
        y_train, y_val = y[i_trn], y[i_val]
        
        lr_clf = LogisticRegression()
        lr_clf.fit(X_train,y_train)
       
        # Predict
        if X.shape[1]==500:
            p_val_ver7[i_val, :] = lr_clf.predict_proba(X[i_val])
            p_tst_ver7 += lr_clf.predict_proba(test) / n_class
        
            
    print("Training has finished")
    print("*"*100)

            
print(f'lr ver7 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver7, axis=1)) * 100:8.4f}%')
print(f'lr ver7 Log Loss (CV): {log_loss(pd.get_dummies(y), p_val_ver7):8.4f}')


Training model for CV #1
Training model for CV #2
Training model for CV #3
Training model for CV #4
Training model for CV #5
Training has finished
****************************************************************************************************
lr ver7 Accuracy (CV):  71.2294%


Exception: Data must be 1-dimensional

## 제출 파일 생성 및 기타 파일 생성

In [ ]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver7
sub[sub.columns] = p_tst_ver7
sub.to_csv(sub_ver7_file)



In [ ]:
# p_val 파일 생성 -> oof

# Ver7
np.savetxt(p_val_ver7_file, p_val_ver7, fmt='%.18f', delimiter=',')


In [ ]:
# p_tst 파일 생성 -> test 

# Ver7
np.savetxt(p_tst_ver7_file, p_tst_ver7, fmt='%.18f', delimiter=',')
